In [88]:
class Sources():
    def __init__(self,insource = [], outsource=[]):
        self.ins = insource
        self.outs = outsource
        self.alldata = {}
        self.sourcelist = None
        updatefirst=True
    
    def setsources(self,insource, outsource):
        self.ins = insource
        self.outs = outsource
    
    def startsetup(self):
        for i in self.ins:
            i.findsource(self.sourcelist)
            i.pushdata()
            self.alldata.update(i.datasource["sdata"])
        for o in self.outs:
            o.findsource(self.sourcelist)
            o.pushdata()
            self.alldata.update(o.datasource["sdata"])
    
    def updateGet(self):
        for i in self.ins:
            i.loaddata()
            self.alldata.update(i.datasource["sdata"])

    def updateSet(self):
        for o in self.outs:
            o.datasource["sdata"]={k: self.alldata[k] for k in o.datasource["sdata"].keys()}
            o.pushdata()

            
class datasource():
    def __init__(self, sourcelabel="", source=None, sdata={}):
        self.datasource = {"sourcelabel":sourcelabel,\
                          "source":source,\
                          "sdata":sdata }
        
    def add(self,data):
        self.datasource["sdata"].update(data)
    
    def clear(self,data):
        self.datasource["sdata"].clear()
    
    def findsource(self, sourcelist):
        self.datasource["source"] = sourcelist[self.datasource["sourcelabel"]]
    
    def updatesource(self,label, sourcelist):
        self.datasource["sourcelabel"]=label
        self.findsource()

    def loaddata(self):
        self.datasource["sdata"]={k: self.datasource["source"].data[k] for k in self.datasource["sdata"].keys()}
    
    def pushdata(self):
        self.datasource["source"].data.update(self.datasource["sdata"])
        

In [144]:
#Basicfunctions:
#-BlockChart

import numpy as np
import cv2
from random import shuffle


def mouseaction(event, x,y,flags,param):
    global MAIN_BC
    
    if event == cv2.EVENT_LBUTTONDOWN:
        MAIN_BC.LBUTTONDOWN(x,y)

    if event == cv2.EVENT_LBUTTONUP:
        MAIN_BC.LBUTTONUP(x,y)

    if event == cv2.EVENT_LBUTTONDBLCLK:
        MAIN_BC.LBUTTONDBLCLK(x,y)
    
    if event == cv2.EVENT_MOUSEMOVE:
        MAIN_BC.MOUSEMOVE(x,y)


class BlockChart():
    def __init__(self, FunctionBlocks, name = "BlockChart", image=np.ones((1600,1800,3), np.uint8)*255):
        global BlockSources
        global sourceDict
        self.image = image
        self.gpos = np.array([0,0])
        self.connections =[]
        self.isRoot =True
        self.parent=None
        self.minimized = False
        #self.currentim = image.copy()
        self.FBs = FunctionBlocks
        self.marg=5
        self.highlightedFB=False
        self.hDX=np.array([0,0])
        self.bgcolor =(255,255,255)
        self.txtcolor = (0,0,0)
        #self.bus=Bus(label="MainBus")
        #self.FBs.append(FunctionBlock(self.bus,size=np.array([120,90]), pos=np.array([0,0])))

        for FB in self.FBs:
            FB.parent=self
            FB.Sources=Sources(BlockSources[FB.function.label]["In"],BlockSources[FB.function.label]["Out"])
            FB.Sources.sourcelist =sourceDict
            FB.Sources.startsetup()
            FB.Resize()
    
    def ResizeChart(self):
        for FB in self.FBs:
            FB.Resize()
        #self.arrangeChart(topcoll = False)
        BB=np.array([9999,0,9999,0])
        for FB in self.FBs:
            bb = FB.BoundingBox()
            if bb[0]<BB[0]: BB[0]=bb[0]
            if bb[1]>BB[1]: BB[1]=bb[1]
            if bb[2]<BB[2]: BB[2]=bb[2]
            if bb[3]>BB[3]: BB[3]=bb[3]
        BB=np.array([BB[0]-self.marg,BB[1]+self.marg,BB[2]-self.marg,BB[3]+self.marg,])
        for FB in self.FBs:
            bb = FB.BoundingBox()
            bb[:2]=bb[:2]-BB[0]
            bb[2:]=bb[2:]-BB[2]
            FB.SetBoundingBox(bb)
        BB[:2]=BB[:2]-BB[0]#+self.marg
        BB[2:]=BB[2:]-BB[2]+12
        self.image=np.ones((BB[3],BB[1],3), np.uint8)
        self.image[:,:,0]=self.bgcolor[0]
        self.image[:,:,1]=self.bgcolor[1]
        self.image[:,:,2]=self.bgcolor[2]
        
        return BB

    
    def deOverlapChart(self):
        arranged = False
        for i in range(len(self.FBs)-1):
            BB=self.FBs[i].BoundingBox()
            for FB in self.FBs[i+1:]:
                bb=FB.BoundingBox()
                ca= ((BB[0]<=bb[0]<=BB[1]) or (BB[0]<=bb[1]<=BB[1])) and \
                     ((BB[2]<=bb[2]<=BB[3]) or (BB[2]<=bb[3]<=BB[3]))
                cb= ((bb[0]<=BB[0]<=bb[1]) or (bb[0]<=BB[1]<=bb[1])) and \
                     ((bb[2]<=BB[2]<=bb[3]) or (bb[2]<=BB[3]<=bb[3]))
                if ca or cb:
                    sx=bb[1]-bb[0]
                    sy=bb[3]-bb[2]
                    #bb[0]=BB[1]+int(sx*0.1)+self.marg
                    #bb[1]=BB[1]+int(sx*1.1)+self.marg
                    bb[2]=BB[3]+int(sy*0.1)+self.marg
                    bb[3]=BB[3]+int(sy*1.1)+self.marg
                    FB.SetBoundingBox(bb)
                    arranged = True
        return arranged      

    def topCollapseChart(self):
        for FB in self.FBs:
            FB.posSet(np.array([FB.pos[1],0]))


    
    def arrangeChart(self, topcoll = True):
        if topcoll:
            self.topCollapseChart()
            arranged = True
        arranged = self.deOverlapChart()
        return arranged        
    
    def PotArrangeChart(self):#not used...
        for j in range(100):
            for i in self.connections:
                pot=2*np.array(potential(i[1].gpos-i[0].gpos),dtype = int)
                i[1].posSet(pot)
                i[0].posSet(-pot)
            k=0
            for FB in self.FBs:
                k+=1
                for FB2 in self.FBs[k:]:
                    pot=2*np.array(potential(FB.gpos-FB2.gpos),dtype = int)
                    FB.posSet(pot)
                    FB2.posSet(-pot)
                    
            
            #    i[0].posSet(np.array((i[0].gpos*2+i[1].gpos)/3-i[0].parent.gpos,dtype = int))
            #    i[1].posSet(np.array((i[1].gpos*2+i[0].gpos)/3-i[1].parent.gpos, dtype=int))
        self.ResizeChart()

    
    def updateChart(self):
        for FB in self.FBs:
            FB.update()       
    
    def DrawChart(self):
        self.connections = []
        self.updateChart()
        #self.ResizeChart()
        im = self.image.copy()
        for FB in self.FBs:
            FB.drawFB(im)
        
        if self.isRoot:
            for i in self.connections:
                self.DrawConn(im,i[0],i[1],i[2], root = True)
            #self.PotArrangeChart()
        return im

    def DrawConn(self, im, obj_from, obj_to, color, root =False):
        pos_from=obj_from.gpos - self.gpos
        pos_to=obj_to.gpos - self.gpos
        if ((all(pos_from == obj_from.pos) and all(pos_to == obj_to.pos)) or root):
            if pos_from[1]<pos_to[1]:
                pos_from[1]+=obj_from.size[1]
                pos_to[0] += obj_to.size[0]*0.55
                pos_from[0] += obj_from.size[0]*0.55
            else:
                pos_to[1]+=obj_to.size[1]
                pos_from[0] += obj_from.size[0]*0.45
                pos_to[0] += obj_to.size[0]*0.45

            cv2.circle(im,(int(pos_from[0]),int(pos_from[1])),3,color,-1)
            cv2.circle(im,(int(pos_to[0]),int(pos_to[1])),3,color,-1)
    
            cv2.line(im,(int(pos_from[0]),int(pos_from[1])),(int(pos_to[0]),int(pos_to[1])),color,1)
            return True
        return False
        
    
    def FBinXY(self,x,y):
        for FB in self.FBs:
            bb=FB.BoundingBox()
            if (bb[0]<=x<=bb[1]) and (bb[2]<=y<=bb[3]):
                return FB
        return False
    
    def LBUTTONDOWN(self,x,y):
        FB = self.FBinXY(x,y)
        if not FB:
            return True
        if FB.LBUTTONDOWN(x,y):
            self.highlightedFB = FB
            self.hDX=self.highlightedFB.pos-np.array([x,y]) 
            return False
        return True

    def LBUTTONUP(self,x,y):
        self.highlightedFB = False
        FB = self.FBinXY(x,y)
        if not FB:
            return True
        return FB.LBUTTONUP(x,y)
       

    
    def LBUTTONDBLCLK(self,x,y):
        FB = self.FBinXY(x,y)
        if not FB:
            return False
        FB.LBUTTONDBLCLK(x,y)
        self.arrangeChart()
        self.ResizeChart()
        return True

    def MOUSEMOVE(self,x,y):
        if self.highlightedFB:
            self.highlightedFB.posSet(np.array([x,y])+self.hDX)
            return False
            #self.ResizeChart()
        FB = self.FBinXY(x,y)
        if not FB:
            return True
        return FB.MOUSEMOVE(x,y)



def potential(dX, rep=False):
    dx=dX[0]
    dy=dX[1]
    r2 = dx*dx + dy*dy
    if rep:
        p= -100.0/(1+r2)
    else:
        p= (r2/128.0-10.0)/(1.0+r2/400.0)/np.sqrt(r2+.0000001)
    return -p*np.array([dx+.1,dy+.1])
        
        

In [22]:
potential(np.array([0,300]))

array([-0.        , -3.06692478])

In [319]:
#-FunctionBlock
#-NullFunction, MainSwitch, Bus, SubChart
        
    
class FunctionBlock():
    def __init__(self, function, name = "NoName", size=np.array([120,90]), pos=np.array([120,90])):
        #basic info
        self.name =name
        self.size =size
        self.pos = pos
        self.gpos = pos
        #connections
        self.function=function
        self.function.parent=self 
        self.parent = None
        self.Sources = None
        #graphics
        self.image = None
        self.useimage = False
        self.textcolor = (0,0,0)
        self.bgcolor=(255,250,250)
        self.linecolor=(0,0,0)
        
    def update(self):
        self.Sources.updateGet()
        if hasattr(self.function, "update"):
            self.function.update()
        self.Sources.updateSet()
    
    def BoundingBox(self):
        return np.array([self.pos[0],self.pos[0]+self.size[0],self.pos[1],self.pos[1]+self.size[1]])
    
    def SetBoundingBox(self,bb):
        pos=np.array([bb[0],bb[2]])
        size=np.array([bb[1]-bb[0],bb[3]-bb[2]])
        self.posSet(pos)
        self.sizeSet(size)
    
    def posSet(self,pos):
        self.pos=pos
        self.gpos=self.parent.gpos + self.pos

    def sizeSet(self,size):
        self.size=size
    
    def Resize(self):
        if hasattr(self.function, "Resize"):
            self.function.Resize()

    def LBUTTONDOWN(self,x,y):
        if hasattr(self.function, "LBUTTONDOWN"):
            x=x-self.pos[0]
            y=y-self.pos[1]
            return self.function.LBUTTONDOWN(x,y)
        return True

    def LBUTTONUP(self,x,y):
        if hasattr(self.function, "LBUTTONUP"):
            x=x-self.pos[0]
            y=y-self.pos[1]
            return self.function.LBUTTONUP(x,y)
        return True

    def MOUSEMOVE(self,x,y):
        if hasattr(self.function, "MOUSEMOVE"):
            x=x-self.pos[0]
            y=y-self.pos[1]
            return self.function.MOUSEMOVE(x,y)
        return True

    def LBUTTONDBLCLK(self,x,y):
        if hasattr(self.function, "LBUTTONDBLCLK"):
            x=x-self.pos[0]
            y=y-self.pos[1]
            return self.function.LBUTTONDBLCLK(x,y)
        return True

    def drawFB(self, im):
        self.posSet(self.pos)
        cv2.rectangle(im, (self.pos[0], self.pos[1]), \
                      (self.pos[0] + self.size[0], self.pos[1] + self.size[1]), self.bgcolor, -1)
        cv2.rectangle(im, (self.pos[0], self.pos[1]), \
                      (self.pos[0] + self.size[0], self.pos[1] + self.size[1]), self.linecolor, 2)
        if hasattr(self.function,"presentData"):
            cv2.putText(im,self.function.presentData(),\
                        (self.pos[0]+int(self.size[0]*.1), self.pos[1] + 30),\
                        cv2.FONT_HERSHEY_SIMPLEX, .25, self.textcolor)
        if hasattr(self.function, "Draw"):
            self.function.Draw(im)
        for i in self.Sources.ins:
            color = (255,150,150)
            self.pushToConnections(i,color)
        for i in self.Sources.outs:
            color = (150,255,150)
            self.pushToConnections(i,color,indirection = False)

    def pushToConnections(self,i,color,indirection = True):
            if not (i.datasource["source"].parent == self):
                ds=i.datasource["source"].parent
                if ds.parent.minimized:
                    ds=ds.parent.parent.parent
                if indirection:
                    self.parent.connections.append([self,ds,color])
                else:
                    self.parent.connections.append([ds,self,color])
                if self.function.type == "SubChart":
                    if i.datasource["source"].parent.parent == self.function.chart:
                        self.parent.connections.pop()
        

class NullFunc():  #Template function
    def __init__(self, label="NullFunc"):
        self.type = "NullFunc"
        self.label =label
        self.parent = None
    
    #def update(self):
    #    pass

    #def Draw(self,im):
    #    pass

    def Resize(self):
        self.parent.sizeSet(np.array([120,35]))    

    #def LBUTTONDOWN(self,x,y):
    #    return True
        
    #def LBUTTONUP(self,x,y):
    #    return True
        
    #def LBUTTONDBLCLK(self,x,y):
    #    return True
    
    #def MOUSEMOVE(self,x,y):
    #    return True
    
    def presentData(self):
        return self.label #tekstiä
    
class OnOff():
    def __init__(self, onstate=False, label ="On/Off"):
        self.type = "On/Off"
        self.label=label
        self.onstate=onstate
        self.parent = None
    
    def update(self):
        self.parent.Sources.alldata["Onstate"] =self.onstate
    
    def swithc(self):
        self.onstate=not self.onstate
    
    def Resize(self):
        self.parent.sizeSet(np.array([120,35]))    

    def Draw(self,im):
        pass

    def LBUTTONDOWN(self,x,y):
        self.onstate = not self.onstate
        return True

    def LBUTTONUP(self,x,y):
        return True
        
        
    def MOUSEMOVE(self,x,y):
        return True
        

    def LBUTTONDBLCLK(self,x,y):
        return True

    def presentData(self):
        if self.onstate:
            state="On"
        else:
            state="Off"
        return self.label+": "+state #tekstiä

class Bus():
    def __init__(self, busdata={"Round":0}, label = "DataBus"):
        self.type = "Bus"
        self.label = label
        self.data = busdata #dictionary
        self.parent = None    
    
    def Resize(self):
        self.parent.size = np.array([500,35])    
 
    def presentData(self):
        return self.label+": "+str(self.data) #tekstiä

    
class SubChart():
    def __init__(self, subchart, label="SubChart"):
        self.type = "SubChart"
        self.label =label
        self.chart=subchart
        self.chart.isRoot =False
        self.chart.parent=self
        #self.minimized = False
        self.parent = None
    
    def update(self):
        self.chart.updateChart()

    def Draw(self,im):
        #self.chart.bgcolor = self.parent.bgcolor
        pos=self.parent.pos
        size=self.parent.size
        self.chart.gpos= self.parent.gpos
        #im[pos[1]:pos[1]+size[1],pos[0]:pos[0]+size[0],:] = self.chart.DrawChart()
        if not self.chart.minimized:
            im[pos[1]:pos[1]+size[1],pos[0]:pos[0]+size[0],:] = self.chart.DrawChart()
            cv2.putText(im,self.presentData(),(pos[0]+2, pos[1] +size[1]-2),cv2.FONT_HERSHEY_SIMPLEX, .5, self.parent.textcolor)
        for i in self.chart.connections:
            if self.chart.minimized:
                if i[0].parent == self.chart:
                    i[0]=self.parent
                if i[1].parent == self.chart:
                    i[1]=self.parent
                if not(i[0]==i[1]):# and not(i[0]==i[1].parent) and not(i[1]==i[0].parent)):
                    self.parent.parent.connections.append(i)
            else:
                self.parent.parent.connections.append(i)

    def Resize(self):
        if not self.chart.minimized:
            self.chart.arrangeChart()
            BB = self.chart.ResizeChart()
            self.parent.sizeSet(np.array([BB[1],BB[3]]))
        else:
            self.parent.sizeSet(np.array([120,35]))

    def LBUTTONDOWN(self,x,y):
        if not self.chart.minimized:
            return self.chart.LBUTTONDOWN(x,y)
        return True

    def LBUTTONUP(self,x,y):
        return self.chart.LBUTTONUP(x,y)

    def MOUSEMOVE(self,x,y):
        return self.chart.MOUSEMOVE(x,y)


    def LBUTTONDBLCLK(self,x,y):
        if self.chart.minimized:
            self.chart.minimized = not self.chart.minimized
            self.Resize()
            return True
        if not self.chart.LBUTTONDBLCLK(x,y):
            self.chart.minimized = not self.chart.minimized
            self.Resize()
            return True
            
    def presentData(self):
        return self.label #tekstiä


In [290]:
np.round(5.232342,2)

5.23

In [318]:
#functions for Environment and Power
class Environment():  #Template function
    def __init__(self, label="Environment"):
        self.type = "Environment"
        self.label =label
        self.parent = None
        self.inside=False
        self.time=0.0 +0.*24*3600# syyspäivä
        self.deltatime=60.0 #sekunteina 
    
    def update(self):
        self.time+=self.deltatime #vaihdetaan logiikan pyytämään aikahyppyyn myöhemmin
        time =12*3600 + self.time
        year=time/(365*24*3600)
        day_of_year=time%(365*24*3600)/(24*3600)
        hour_of_day=(time%(24*3600))/3600
        minute=np.floor((time%3600)/60)
        second= np.floor((time%60)/60)
        lightness = max((np.cos(hour_of_day/12*np.pi)+\
                        0.5*np.cos(day_of_year/365*2*np.pi))+\
                        np.average(np.random.random(5)-0.3),0)
        self.parent.bgcolor=tuple([min(k*lightness*255,255)for k in [.8,1,1.3]])
        self.parent.Sources.alldata["Lightness"] =lightness

    #def Draw(self,im):
    #    pass

    def Resize(self):
        self.parent.sizeSet(np.array([120,35]))    

    #def LBUTTONDOWN(self,x,y):
    #    return True
        
    #def LBUTTONUP(self,x,y):
    #    return True
        
    #def LBUTTONDBLCLK(self,x,y):
    #    return True
    
    #def MOUSEMOVE(self,x,y):
    #    return True
    
    def presentData(self):
        return self.label+": day#:"+str(self.time/(24*3600)) #tekstiä


class Photovoltaic():  #Template function
    def __init__(self, label="SolarCell"):
        self.type = "Photovoltaic"
        self.label =label
        self.parent = None
        self.deltatime=60.0 #sekunteina 
        self.V_PV_Out = 0.0
    
    def update(self):
        dt=self.deltatime #vaihdetaan logiikan pyytämään aikahyppyyn myöhemmin
        L=self.parent.Sources.alldata["Lightness"]
        self.V_PV_Out = L*3.0
        self.parent.bgcolor=tuple([min(k*self.V_PV_Out*50+kk,255)for k,kk in zip([0,1,-1],[100,100,255])])
        self.parent.Sources.alldata["V_PV_Out"] =self.V_PV_Out

    #def Draw(self,im):
    #    pass

    def Resize(self):
        self.parent.sizeSet(np.array([120,35]))    

    #def LBUTTONDOWN(self,x,y):
    #    return True
        
    #def LBUTTONUP(self,x,y):
    #    return True
        
    #def LBUTTONDBLCLK(self,x,y):
    #    return True
    
    #def MOUSEMOVE(self,x,y):
    #    return True
    
    def presentData(self):
        return self.label+": V_out"+str(self.V_PV_Out) #tekstiä

class Supercaps():  #Template function
    def __init__(self, label="Supercaps"):
        self.type = "Supercap"
        self.label =label
        self.parent = None
        self.deltatime=60.0 #sekunteina 
        self.E_SC=0.0
    
    def update(self):
        dt=self.deltatime #vaihdetaan logiikan pyytämään aikahyppyyn myöhemmin
        V_PV=self.parent.Sources.alldata["V_PV_Out"]
        self.E_SC+=(V_PV*.015 - .02)*dt # P=UI, E=P*dt, 
        self.E_SC = min(self.E_SC,1000)
        self.E_SC = max(self.E_SC,0)
        self.parent.bgcolor=tuple([min(k*self.E_SC/5+kk,255)for k,kk in zip([0,1,-1],[100,100,255])])
        self.parent.Sources.alldata["E_SC"] =self.E_SC

    #def Draw(self,im):
    #    pass

    def Resize(self):
        self.parent.sizeSet(np.array([120,35]))    

    #def LBUTTONDOWN(self,x,y):
    #    return True
        
    #def LBUTTONUP(self,x,y):
    #    return True
        
    #def LBUTTONDBLCLK(self,x,y):
    #    return True
    
    #def MOUSEMOVE(self,x,y):
    #    return True
    
    def presentData(self):
        return self.label+": E:"+str(self.E_SC) #tekstiä

class Monitor():  #Template function
    def __init__(self, label="Monitor"):
        self.type = "Monitor"
        self.label =label
        self.parent = None
        self.deltatime=60.0 #sekunteina 
        self.MonitorData =[]
        self.monitorImage = None
        self.h=150
        self.w=800
        self.sourcelabels =["V_PV","E_SC","Lightness"]
        self.minimized =False
    
    def update(self):
        dt=self.deltatime #vaihdetaan logiikan pyytämään aikahyppyyn myöhemmin
        V_PV=self.parent.Sources.alldata["V_PV_Out"]
        E_SC=self.parent.Sources.alldata["E_SC"]
        lightness = self.parent.Sources.alldata["Lightness"]
        self.MonitorData.append([V_PV,E_SC,lightness]) 

    def Draw(self,im):
        if not self.minimized:
            data=np.array(self.MonitorData[-self.w:])
            mxd=[0,0,0]
            for i in range(len(data[0])):
                mxd[i]=max(data[:,i])
                #mni=min(data[:,i])
                if mxd[i]>0:
                    data[:,i]=self.h-data[:,i]/mxd[i]*self.h
            #if len(data)>300: data[:,:300]=data[:,-300:]
            img=np.zeros((self.h,self.w,3), np.uint8)
        
            for j in range(len(data[0])):
                #for i
                #cv2.polylines(img, np.int32([points]), 1, (255,255,255))
                for i in range(len(data)-1):        
                    cv2.line(img,(i,int(data[i,j])),(i+1,int(data[i+1,j])),(j*70,255-j*70,j*70),1)
                    cv2.putText(img,self.sourcelabels[j]+" MX: "+str(np.round(mxd[j],2)),\
                        (5,10+j*15),\
                        cv2.FONT_HERSHEY_SIMPLEX, .25, (j*70,255-j*70,j*70) )

            im[self.parent.pos[1]:self.parent.pos[1]+self.h,self.parent.pos[0]:self.parent.pos[0]+self.w]=img
        

    def Resize(self):
        self.parent.textcolor = (255,200,200)
        if self.minimized:
            self.parent.sizeSet(np.array([120,90]))
            self.parent.bgcolor=(50,0,0)
        else:
            self.parent.sizeSet(np.array([self.w,self.h]))    

    #def LBUTTONDOWN(self,x,y):
    #    return True
        
    #def LBUTTONUP(self,x,y):
    #    return True
        
    def LBUTTONDBLCLK(self,x,y):
        self.minimized=not self.minimized
        return True
    
    #def MOUSEMOVE(self,x,y):
    #    return True
    
    def presentData(self):
        return self.label #tekstiä

    

In [202]:
a=[[2,3,4],[2,5,6],[9,2,3]]
b=np.array(a)
c=np.arange(3)
c[-2:]

array([1, 2])

In [149]:
global BlockSources
BlockSources ={\
    "MainBus":{\
        "In":[datasource(sourcelabel = "MainBus",sdata={"Self":True})],\
        "Out":[datasource(sourcelabel = "MainBus",sdata={"Self":True})]\
    },\
    "PowerBus":{\
        "In":[datasource(sourcelabel = "PowerBus",sdata={"Self":True})],\
        "Out":[datasource(sourcelabel = "PowerBus",sdata={"Self":True})]\
    },\
    "Mains":{\
        "In":[datasource(sourcelabel = "PowerBus",sdata={"Self":True})],\
        "Out":[datasource(sourcelabel = "PowerBus",sdata={"Onstate":False})]\
    },\
    "Solar cell":{\
        "In":[datasource(sourcelabel = "EnvironmentData",sdata={"Lightness":0.0})],\
        "Out":[datasource(sourcelabel = "PowerBus",sdata={"P_PV_Out":0.0,"V_PV_Out":0.0})]\
    },\
    "Long Term Battery":{\
        "In":[datasource(sourcelabel = "EnvironmentData",sdata={"Temperature":10.0})],\
        "Out":[datasource(sourcelabel = "PowerBus",sdata={"VBatt":3.1})]\
    },\
    "Supercaps":{\
        "In":[datasource(sourcelabel = "PowerBus",sdata={"P_SC_Req":0.0,"P_In_SC":0.0,"V_PV_Out":0.0})],\
        "Out":[datasource(sourcelabel = "PowerBus",sdata={"E_SC":0.0,"P_Out_SC":0.0})]\
                },\
    "Harvester circuitry":{\
        "In":[datasource(sourcelabel = "PowerBus",\
                            sdata={\
                                    "V_SC":0.0,"P_Out_SC":0.0,\
                                    "VBatt":3.1,"Onstate":False,\
                                    "P_PV_Out":0.0,"V_PV_Out":0.0\
                                    }),\
               datasource(sourcelabel = "MainBus",\
                            sdata={\
                                    "PeakPowerRequest":False,\
                                   "PowerLowAlert":False,
                                    })],\
        "Out":[datasource(sourcelabel = "PowerBus",\
                            sdata={\
                                    "P_SC_Req":0.0,"P_In_SC":0.0,\
                                   "P_To_Reg":0.0, "V_to_Reg":0.0\
                                    }),
               datasource(sourcelabel = "MainBus",\
                            sdata={\
                                    "PowerCapacity":0.0,"PowerLowAlert":False,\
                                   "PowerShuttingDown":False\
                                    })]\
    },\
        "Voltage regulation":{\
        "In":[datasource(sourcelabel = "PowerBus",sdata={"P_To_Reg":0.0, "V_to_Reg":0.0})],\
        "Out":[datasource(sourcelabel = "PowerBus",sdata={"V_SC":0.0,"P_Out_SC":0.0}),\
               datasource(sourcelabel = "MainBus",sdata={"P_TOT_Out":0.0,"V_TOT_Out":False})]\
    },\
        "Power block":{\
        "In":[datasource(sourcelabel = "PowerBus",sdata={"Block":"PowerBlock"})],\
        "Out":[datasource(sourcelabel = "PowerBus",sdata={"Block":"PowerBlock"})]\
    },\
    "EnvironmentData":{\
        "In":[datasource(sourcelabel = "EnvironmentData",sdata={"Self":True})],\
        "Out":[datasource(sourcelabel = "EnvironmentData",sdata={"Self":True})]\
    },\
        "Environment":{\
        "In":[datasource(sourcelabel = "EnvironmentData",sdata={"Temperature":5.0})],\
        "Out":[datasource(sourcelabel = "EnvironmentData",sdata={"Temperature":5.0,"Lightness":0.0})]\
    },\
        "Environment block":{\
        "In":[datasource(sourcelabel = "EnvironmentData",sdata={"Block":"EnvironmentBlock"})],\
        "Out":[datasource(sourcelabel = "EnvironmentData",sdata={"Block":"EnvironmentBlock"})]\
    },\
        "MainMonitor":{\
        "In":[datasource(sourcelabel = "EnvironmentData",sdata={"Lightness":0.0}),\
             datasource(sourcelabel = "PowerBus",sdata={"E_SC":0.0,"V_PV_Out":0.0})],\
        "Out":[datasource(sourcelabel = "MainBus",sdata={"Monitor":True})]\
    }}

In [324]:
global sourceDict
sourceDict ={}
#Power block###########################
FB_power=[]
data={"Round":0}
fb1=Bus(busdata=data,label="PowerBus")
sourceDict.update({"PowerBus":fb1})
FB_power.append(FunctionBlock(fb1,size=np.array([120,90]), pos=np.array([0,0])))

f=OnOff(True, label ="Mains")
FB1=FunctionBlock(f,size=np.array([120,90]), pos=np.array([0,0]))
FB_power.append(FB1)

f1=Photovoltaic(label ="Solar cell")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,0]))
FB.bgcolor=(200,200,255)
FB_power.append(FB)

f1=NullFunc(label ="Long Term Battery")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,99]))
FB.bgcolor=(230,230,255)
FB_power.append(FB)

f1=Supercaps(label ="Supercaps")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(200,200,255)
FB_power.append(FB)

f1=NullFunc(label ="Harvester circuitry")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,299]))
FB.bgcolor=(200,255,255)
FB_power.append(FB)

f1=NullFunc(label ="Voltage regulation")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,399]))
FB.bgcolor=(200,255,255)
FB_power.append(FB)



#Logics block#########################
FB_logic=[]
data={"Round":0}
fb1=Bus(busdata=data,label="SensorBus")
sourceDict.update({"SensorBus":fb1})
FB_logic.append(FunctionBlock(fb1,size=np.array([120,90]), pos=np.array([0,0])))

f1=NullFunc(label ="Microcontroller")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,0]))
FB.bgcolor=(200,255,255)
FB_logic.append(FB)

f1=NullFunc(label ="Sensors")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,99]))
FB.bgcolor=(200,255,255)
FB_logic.append(FB)

f1=NullFunc(label ="Indicator")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(200,255,255)
FB_logic.append(FB)



#Radio block##########################
FB_radio=[]
data={"Round":0}
fb1=Bus(busdata=data,label="RadioBus")
sourceDict.update({"RadioBus":fb1})
FB_radio.append(FunctionBlock(fb1,size=np.array([120,90]), pos=np.array([0,0])))

f1=NullFunc(label ="Communication radio")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(200,255,255)
FB_radio.append(FB)

f1=NullFunc(label ="Positioning radio")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(230,255,230)
FB_radio.append(FB)

f1=NullFunc(label ="NFC")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(230,255,230)
FB_radio.append(FB)

f1=NullFunc(label ="Antenna")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(200,255,255)
FB_radio.append(FB)

f1=NullFunc(label ="Posit. Antenna")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(230,255,230)
FB_radio.append(FB)


FB_gateway=[]

f1=NullFunc(label ="Gateway")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(200,255,255)
FB_gateway.append(FB)

data={"Round":0}
fb1=Bus(busdata=data,label="RadioChannel")
sourceDict.update({"RadioChannel":fb1})
FB_gateway.append(FunctionBlock(fb1,size=np.array([12,9]), pos=np.array([0,0])))




#Environment Block
FB_env =[]
f1=Environment(label ="Environment")
FB=FunctionBlock(f1,size=np.array([120,90]), pos=np.array([0,199]))
FB.bgcolor=(200,255,255)
FB_env.append(FB)

data={"Round":0}
fb1=Bus(busdata=data,label="EnvironmentData")
sourceDict.update({"EnvironmentData":fb1})
FB_env.append(FunctionBlock(fb1,size=np.array([12,9]), pos=np.array([0,0])))

#MAIN SECTION
FBs=[]
data={"Round":0}
fb1=Bus(busdata=data,label="MainBus")
sourceDict.update({"MainBus":fb1})
FBs.append(FunctionBlock(fb1,size=np.array([12,9]), pos=np.array([0,0])))

fb1=Monitor(label="MainMonitor")
sourceDict.update({"MainMonitor":fb1})
FBs.append(FunctionBlock(fb1,size=np.array([12,9]), pos=np.array([0,0])))


#Block charts
BC_power=BlockChart(FB_power)
BC_env=BlockChart(FB_env)
'''
BC_gateway=BlockChart(FB_gateway)
ss=SubChart(BC_gateway,label = "GateWay block")
ss.chart.bgcolor=(255,255,200)
F=FunctionBlock(ss,size=np.array([777,777]), pos=np.array([0,0]))
F.linecolor=(255,66,66)
FB_radio.append(F)



BC_radio=BlockChart(FB_radio)
BC_logic=BlockChart(FB_logic)
'''

'\nBC_gateway=BlockChart(FB_gateway)\nss=SubChart(BC_gateway,label = "GateWay block")\nss.chart.bgcolor=(255,255,200)\nF=FunctionBlock(ss,size=np.array([777,777]), pos=np.array([0,0]))\nF.linecolor=(255,66,66)\nFB_radio.append(F)\n\n\n\nBC_radio=BlockChart(FB_radio)\nBC_logic=BlockChart(FB_logic)\n'

In [325]:

#Blockchart

ss=SubChart(BC_power,label = "Power block")
ss.chart.bgcolor=(200,240,255)
F=FunctionBlock(ss,size=np.array([777,777]), pos=np.array([0,0]))
F.linecolor=(255,66,66)
FBs.append(F)
'''
ss=SubChart(BC_logic,label = "Logic block")
ss.chart.bgcolor=(150,255,150)
F=FunctionBlock(ss,size=np.array([777,777]), pos=np.array([0,0]))
F.linecolor=(66,66,255)
FBs.append(F)

ss=SubChart(BC_radio,label = "Radio block")
ss.chart.bgcolor=(150,255,150)
F=FunctionBlock(ss,size=np.array([777,777]), pos=np.array([0,0]))
F.linecolor=(66,255,255)
FBs.append(F)
'''
ss=SubChart(BC_env,label = "Environment block")
ss.chart.bgcolor=(150,255,255)
F=FunctionBlock(ss,size=np.array([777,777]), pos=np.array([0,0]))
F.linecolor=(66,255,66)
FBs.append(F)



BC=BlockChart(FBs)
MAIN_BC =BC

BC.arrangeChart()
BC.ResizeChart()
BC.arrangeChart()
BC.ResizeChart()
#image=np.ones((600,800,3), np.uint8)*255
cv2.startWindowThread()
cv2.namedWindow('BlockChart')
cv2.setMouseCallback('BlockChart',mouseaction)
while True:
    im=BC.DrawChart()
    cv2.imshow("BlockChart",im)
    inp = cv2.waitKey(1)
    if inp==97:
        MAIN_BC.ResizeChart()
        MAIN_BC.arrangeChart()
        MAIN_BC.ResizeChart()
    if inp==27:
        break
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [185]:
len(fb1.MonitorData[1])

3

In [ ]:
BC.unDrawed

In [ ]:
for i in BC_power.unDrawed:
    posb=i[1].parent.gpos - BC_power.gpos
    posf=i[0].gpos - BC_power.gpos
    print(posb,posf, BC.gpos,BC_power.gpos)


In [ ]:
dir(cv2)